In [14]:
import csv

# Function to count occurrences of a specific string in a CSV file
def count_error_occurrences(file_path, target_string):
    error_count = 0
    with open(file_path, 'r', newline='') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            for cell in row:
                if target_string in cell:
                    error_count += 1
    return error_count

# Specify the path to your parsed CSV file
csv_file_path = 'parsed.csv'

# Specify the target string you want to count
target_string = 'Error parsing the query.'

# Call the function to count occurrences
error_count = count_error_occurrences(csv_file_path, target_string)

# Print the result
print(f"Number of '{target_string}' occurrences: {error_count}")


Number of 'Error parsing the query.' occurrences: 291


# Global variations

In [ ]:
import csv

def normalize_sparql_queries(input_csv_path, output_csv_path):
    global_var_counter = 1  # Initialize the global variable counter

    with open(input_csv_path, 'r') as input_csv, open(output_csv_path, 'w', newline='') as output_csv:
        csv_reader = csv.reader(input_csv)
        csv_writer = csv.writer(output_csv)

        for row in csv_reader:
            if not row:
                continue  # Skip empty rows

            sparql_query = row[1]  # Assuming the SPARQL query is in the first column

            # Extract variables from the SPARQL query using regular expressions
            variables = re.findall(r'\?([a-zA-Z_][a-zA-Z0-9_]*)', sparql_query)

            # Create a mapping of old variables to new numerated variables
            variable_mapping = {}
            for variable in variables:
                if variable not in variable_mapping:
                    variable_mapping[variable] = f'?var{global_var_counter}'
                    global_var_counter += 1

            # Replace variables in the SPARQL query with numerated variables
            for old_variable, new_variable in variable_mapping.items():
                sparql_query = sparql_query.replace(f'?{old_variable}', new_variable)

            # Write the normalized query to the output CSV
            csv_writer.writerow([sparql_query])

if __name__ == "__main__":
    input_csv_path = "unique_normalised_12_9_2023.csv"  # Replace with the path to your input CSV file
    output_csv_path = "normalized_queries.csv"  # Replace with the desired output CSV file

    normalize_sparql_queries(input_csv_path, output_csv_path)


In [ ]:
import pandas as pd

# Set the maximum column width to None to display the entire content of each cell
pd.set_option('display.max_colwidth', None)

df = pd.read_csv('normalized_queries.csv', lineterminator='\n', dtype=str, header=None)

print(len(df))
df.head(10)

get a sample

In [24]:
import pandas as pd

# Define the input and output file paths
input_csv_file = 'find_bug_triples.csv'  # Replace with your input CSV file path
output_csv_file = 't.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(input_csv_file)

# Get the first 1000 rows, including the header
sample_df = df.head(10 )

# Save the sample DataFrame to an output CSV file
sample_df.to_csv(output_csv_file, index=False)

print(f"{len(sample_df)} rows have been saved to {output_csv_file}.")


10 rows have been saved to t.csv.


# get triples

In [ ]:
import csv
import json

# Load your CSV data from the input file
input_csv_file = 'parsed.csv'
output_csv_file = 'find_bug_triples.csv'

# Create a list to store the triples
triples = []

# Define a function to extract triples from the parse tree
def extract_triples(parse_tree):
    if "triples" in parse_tree:
        for triple in parse_tree["triples"]:
            try:
                subject = triple["subject"]["value"]
                predicate = triple["predicate"]["value"]
                obj = triple["object"]["value"]
                triples.append((subject, predicate, obj))
            except KeyError as e:
                print(f"Error extracting triple: {e}")
                print("Offending triple:", triple)
    
    # Recursively call the function on child nodes
    for key, value in parse_tree.items():
        if isinstance(value, dict):
            extract_triples(value)
        elif isinstance(value, list):
            for item in value:
                if isinstance(item, dict):
                    extract_triples(item)

# Read the CSV file and process each row
with open(input_csv_file, 'r') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        # Check if there is content in the second column and it doesn't contain the error message
        if len(row) > 1 and row[1] and "Error parsing the query" not in row[1]:
            # Extract triples from the content of the second column
            content = row[1]
            try:
                parse_tree = json.loads(content)
                extract_triples(parse_tree)
            except json.JSONDecodeError:
                # Handle invalid JSON if needed
                pass

# Write the extracted triples to the output CSV file
with open(output_csv_file, 'w', newline='') as csvfile:
    fieldnames = ['subject', 'predicate', 'object']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # Write the header
    writer.writeheader()

    # Write the triples
    for triple in triples:
        writer.writerow({'subject': triple[0], 'predicate': triple[1], 'object': triple[2]})

print("Triples extracted and written to", output_csv_file)


In [1]:
# Define the input and output file names
input_file = 'find_bug_triples.csv'
output_file = 'filtered_data.csv'

# Open the input data file for reading and the output file for writing
with open(input_file, 'r') as input_file, open(output_file, 'w') as output_file:
    for line in input_file:
        # Split the line into components using tab (',') as the delimiter
        components = line.strip().split(',')

        # Check if the line has exactly three components (subject, predicate, object)
        if len(components) == 3:
            # Write complete triples to the output file
            output_file.write(line)


# entity extraction from triples:
if entity start with var do not write it in output file.

if entity start with g_ do not write it in output file.

if a entity has this substring "nonsensical" do not write it in output file.

only write unique entities in output csv

entities with spaces like are encoded to improve clustering "DrugBank drugbank_vocabulary:Indication"

2177101 entities redeuced to 30918


In [1]:
import csv

input_file_name = 'filtered_data.csv'
output_file_name = 'extracted_entities.csv'

subjects_and_objects = set()  # Using a set to store unique entities

with open(input_file_name, mode ='r') as input_file:
    csvReader = csv.reader(input_file)
    
    # Skip header if there's any
    next(csvReader, None)

    for row in csvReader:
        # Assuming that the subject is the first element and object is the third element in the row
        subject, predicate, object_ = row

        if not subject.startswith('var') and not subject.startswith('g_') and 'nonsensical' not in subject:
            subjects_and_objects.add(subject)
            
        if not object_.startswith('var') and not object_.startswith('g_') and 'nonsensical' not in object_:
            subjects_and_objects.add(object_)

# Write the extracted subjects and objects to an output CSV file
with open(output_file_name, mode ='w', newline='') as output_file:
    csvWriter = csv.writer(output_file)
    
    # Writing header
    csvWriter.writerow(['Entity'])
    
    # Writing data
    for entity in subjects_and_objects:
        csvWriter.writerow([entity])

print(f"Extracted entities have been saved to {output_file_name}")


Extracted entities have been saved to extracted_entities.csv


In [2]:
import pandas as pd

# Set the maximum column width to None to display the entire content of each cell
pd.set_option('display.max_colwidth', None)

df = pd.read_csv('extracted_entities.csv', lineterminator='\n', dtype=str, header=None)

print(len(df))
df.head(100)

30920


,0
0,Entity\r
1,\r
2,http://purl.org/goodrelations/v1#Offering\r
3,SLC9A4\r
4,http://bio2rdf.org/kegg:K03908\r
...,...
95,http://bio2rdf.org/wormbase:WBGene00198889\r
96,http://dbpedia.org/resource/Nortriptyline\r
97,SUCLG1\r
98,http://bio2rdf.org/go.ref_vocabulary:Resource\r


# query type of entities 

In [ ]:
!pip install SPARQLWrapper


In [9]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("http://bio2rdf.org/sparql")  # Replace with the actual Bio2RDF SPARQL endpoint
sparql.setQuery("""
                PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
                PREFIX dcterms: <http://purl.org/dc/terms/>
                SELECT ?type 
                WHERE {{ 
                       ?s dcterms:title "GPX3" ;
                                rdf:type ?type .
                                                  }}
            """)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    print(result["type"]["value"])


http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#C43568


In [5]:
import csv
import re
from SPARQLWrapper import SPARQLWrapper, JSON

sparql_endpoint = "http://bio2rdf.org/sparql"

sparql = SPARQLWrapper(sparql_endpoint)
sparql.setReturnFormat(JSON)

def is_url(string):
    url_regex = re.compile(
        r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    )
    return re.match(url_regex, string) is not None

def execute_query_and_write_results(entity, query):
    try:
        sparql.setQuery(query)
        results = sparql.query().convert()
        types = set()
        titles = set()

        for result in results["results"]["bindings"]:
            type_value = result.get("type", {}).get("value")
            title_value = result.get("title", {}).get("value")
            
            if type_value:
                types.add(type_value)
            
            if title_value:
                titles.add(title_value)

        for type_value in types:
            types_file.write(type_value + '\n')

        for title_value in titles:
            titles_file.write(title_value + '\n')

        if not types:
            without_type_file.write(entity + '\n')

    except Exception as e:
        print(f"An error occurred while processing the entity {entity}: {e}")

with open('s1.csv', mode ='r') as file, \
     open('stypes.txt', 'w') as types_file, \
     open('stitles.txt', 'w') as titles_file, \
     open('swithout_type.txt', 'w') as without_type_file:

    csv_reader = csv.reader(file)
    for row in csv_reader:
        entity = row[0].strip()

        if is_url(entity):
            query = f"""
                PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
                PREFIX dcterms: <http://purl.org/dc/terms/>

                SELECT DISTINCT ?type ?title
                WHERE {{
                    <{entity}> rdf:type ?type .
                     
                }}
            """
        else:
            query = f"""
                PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
                PREFIX dcterms: <http://purl.org/dc/terms/>
                SELECT ?type 
                WHERE {{ 
                       ?s dcterms:title "{entity}" ;
                                rdf:type ?type .
                                                  }}
            """

        execute_query_and_write_results(entity, query)


In [4]:
# Open the types.txt file and read the types
with open('stypes.txt', 'r') as file:
    types = file.readlines()

# Use a set to get unique types
unique_types = set([type.strip() for type in types])

# Write the unique types to types.csv
with open('stypes.csv', 'w', newline='') as file:
    for type in unique_types:
        file.write(type + '\n')


# get types but if the entity is a type return itself and other types

In [7]:
import csv
import re
from SPARQLWrapper import SPARQLWrapper, JSON
 
sparql_endpoint = "http://bio2rdf.org/sparql"

sparql = SPARQLWrapper(sparql_endpoint)
sparql.setReturnFormat(JSON)

def is_url(string):
    url_regex = re.compile(
        r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    )
    return re.match(url_regex, string) is not None

def execute_query_and_write_results(entity, query, is_entity_url):
    try:
        sparql.setQuery(query)
        results = sparql.query().convert()
        types = set()
        has_results = bool(results["results"]["bindings"])  # Check if the result set is not empty

        for result in results["results"]["bindings"]:
            type_value = result.get("type", {}).get("value")
            
            if type_value:
                if type_value in ["http://www.w3.org/2002/07/owl#Class", "http://www.w3.org/2000/01/rdf-schema#Class"]:
                    types.add(entity)
                else:
                    types.add(type_value)
 
        if has_results and not is_entity_url:
            titles_file.write(entity + '\n')

        for type_value in types:
            types_file.write(type_value + '\n')

        if not types:
            without_type_file.write(entity + '\n')

    except Exception as e:
        print(f"An error occurred while processing the entity {entity}: {e}")

with open('extracted_entities.csv', mode ='r') as file, \
     open('types1.txt', 'w') as types_file, \
     open('titles1.txt', 'w') as titles_file, \
     open('without_type1.txt', 'w') as without_type_file:

    csv_reader = csv.reader(file)
    for row in csv_reader:
        entity = row[0].strip()
        is_entity_url = is_url(entity)

        if is_entity_url:
            query = f"""
                PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
                PREFIX dcterms: <http://purl.org/dc/terms/>

                SELECT DISTINCT ?type  
                WHERE {{
                    <{entity}> rdf:type ?type .
                                    }}
            """
        else:
            query = f"""
                PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
                PREFIX dcterms: <http://purl.org/dc/terms/>
                SELECT ?type 
                WHERE {{ 
                       ?s dcterms:title "{entity}" ;
                                rdf:type ?type .
                }}
            """

        execute_query_and_write_results(entity, query, is_entity_url)
        

## query extracted entity againt only bio2rdf graphs

In [ ]:
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON
from urllib.parse import urlparse
import time
# Load entities from CSV file
entities_df = pd.read_csv('extracted_entities.csv')
entities = entities_df.iloc[:, 0].tolist()

# Define the Bio2RDF SPARQL endpoint
sparql_endpoint = "http://bio2rdf.org/sparql"

# Define the graphs to query
graphs = [
"http://bio2rdf.org/sgd_resource:bio2rdf.dataset.sgd.R3",
"http://bio2rdf.org/taxonomy_resource:bio2rdf.dataset.taxonomy.R3",
"http://bio2rdf.org/homologene_resource:bio2rdf.dataset.homologene.R3",
"http://bio2rdf.org/interpro_resource:bio2rdf.dataset.interpro.R3",
"http://bio2rdf.org/bioportal_resource:bio2rdf.dataset.bioportal.R3",
"http://bio2rdf.org/clinicaltrials_resource:bio2rdf.dataset.clinicaltrials.R3",
"http://bio2rdf.org/kegg_resource:bio2rdf.dataset.kegg.R3",
"http://bio2rdf.org/pharmgkb_resource:bio2rdf.dataset.pharmgkb.R3",
"http://bio2rdf.org/hgnc_resource:bio2rdf.dataset.hgnc.R3",
"http://bio2rdf.org/mesh_resource:bio2rdf.dataset.mesh.R3",
"http://bio2rdf.org/omim_resource:bio2rdf.dataset.omim.R3",
"http://bio2rdf.org/sider_resource:bio2rdf.dataset.sider.R3",
"http://bio2rdf.org/apo_resource:bio2rdf.dataset.apo.R3",
"http://bio2rdf.org/ctd_resource:bio2rdf.dataset.ctd.R3",
"http://bio2rdf.org/go_resource:bio2rdf.dataset.go.R3",
"http://bio2rdf.org/hp_resource:bio2rdf.dataset.hp.R3",
"http://bio2rdf.org/drugbank_resource:bio2rdf.dataset.drugbank.R3",
"http://bio2rdf.org/mgi_resource:bio2rdf.dataset.mgi.R3",
"http://bio2rdf.org/goa_resource:bio2rdf.dataset.goa.R3",
"http://bio2rdf.org/ndc_resource:bio2rdf.dataset.ndc.R3",
"http://bio2rdf.org/wormbase_resource:bio2rdf.dataset.wormbase.R3",
"http://bio2rdf.org/lsr_resource:bio2rdf.dataset.lsr.R3",
"http://bio2rdf.org/affymetrix_resource:bio2rdf.dataset.affymetrix.R3",
"http://bio2rdf.org/ncbigene_resource:bio2rdf.dataset.ncbigene.R3",
"http://bio2rdf.org/eco_resource:bio2rdf.dataset.eco.R3",
"http://bio2rdf.org/irefindex_resource:bio2rdf.dataset.irefindex.R3"]

# Initialize SPARQLWrapper
sparql = SPARQLWrapper(sparql_endpoint)

# Function to format the entity for the SPARQL query
def format_entity(entity):
    try:
        result = urlparse(entity)
        if all([result.scheme, result.netloc]):
            return f"<{entity}>"  # URL
    except:
        pass
    return f"\"{entity}\""  # Literal

# Function to check if an entity exists in a graph
def entity_exists_in_graph(entity, graph):
    formatted_entity = format_entity(entity)
    
    # Check as subject
    sparql.setQuery(f"""
    ASK WHERE {{
        GRAPH <{graph}> {{
            {formatted_entity} ?p ?o.
        }}
    }}
    """)
    sparql.setReturnFormat(JSON)
    exists_as_subject = sparql.query().convert()['boolean']

    # Adding a delay to reduce load on the server
    # time.sleep(1)  # Delay for 1 second

    if exists_as_subject:
        return True

    # Check as object
    sparql.setQuery(f"""
    ASK WHERE {{
        GRAPH <{graph}> {{
            ?s ?p {formatted_entity}.
        }}
    }}
    """)
    sparql.setReturnFormat(JSON)
    exists_as_object = sparql.query().convert()['boolean']

    # Adding a delay to reduce load on the server
    # time.sleep(1)  # Delay for 1 second

    return exists_as_object

# Check entities and write valid ones to a new CSV file
valid_entities = []
for entity in entities:
    found = False
    for graph in graphs:
        if entity_exists_in_graph(entity, graph):
            valid_entities.append(entity)
            found = True
            break
        # Adding a delay to reduce load on the server
        # time.sleep(1)  # Delay for 1 second
    if found:
        continue

# Write valid entities to CSV
pd.DataFrame(valid_entities, columns=['Valid Entities']).to_csv('10_10_valid_entities.csv', index=False)


In [ ]:
import csv
import re
from SPARQLWrapper import SPARQLWrapper, JSON
 
sparql_endpoint = "http://bio2rdf.org/sparql"

sparql = SPARQLWrapper(sparql_endpoint)
sparql.setReturnFormat(JSON)

graphs = [
"http://bio2rdf.org/sgd_resource:bio2rdf.dataset.sgd.R3",
"http://bio2rdf.org/taxonomy_resource:bio2rdf.dataset.taxonomy.R3",
"http://bio2rdf.org/homologene_resource:bio2rdf.dataset.homologene.R3",
"http://bio2rdf.org/interpro_resource:bio2rdf.dataset.interpro.R3",
"http://bio2rdf.org/bioportal_resource:bio2rdf.dataset.bioportal.R3",
"http://bio2rdf.org/clinicaltrials_resource:bio2rdf.dataset.clinicaltrials.R3",
"http://bio2rdf.org/kegg_resource:bio2rdf.dataset.kegg.R3",
"http://bio2rdf.org/pharmgkb_resource:bio2rdf.dataset.pharmgkb.R3",
"http://bio2rdf.org/hgnc_resource:bio2rdf.dataset.hgnc.R3",
"http://bio2rdf.org/mesh_resource:bio2rdf.dataset.mesh.R3",
"http://bio2rdf.org/omim_resource:bio2rdf.dataset.omim.R3",
"http://bio2rdf.org/sider_resource:bio2rdf.dataset.sider.R3",
"http://bio2rdf.org/apo_resource:bio2rdf.dataset.apo.R3",
"http://bio2rdf.org/ctd_resource:bio2rdf.dataset.ctd.R3",
"http://bio2rdf.org/go_resource:bio2rdf.dataset.go.R3",
"http://bio2rdf.org/hp_resource:bio2rdf.dataset.hp.R3",
"http://bio2rdf.org/drugbank_resource:bio2rdf.dataset.drugbank.R3",
"http://bio2rdf.org/mgi_resource:bio2rdf.dataset.mgi.R3",
"http://bio2rdf.org/goa_resource:bio2rdf.dataset.goa.R3",
"http://bio2rdf.org/ndc_resource:bio2rdf.dataset.ndc.R3",
"http://bio2rdf.org/wormbase_resource:bio2rdf.dataset.wormbase.R3",
"http://bio2rdf.org/lsr_resource:bio2rdf.dataset.lsr.R3",
"http://bio2rdf.org/affymetrix_resource:bio2rdf.dataset.affymetrix.R3",
"http://bio2rdf.org/ncbigene_resource:bio2rdf.dataset.ncbigene.R3",
"http://bio2rdf.org/eco_resource:bio2rdf.dataset.eco.R3",
"http://bio2rdf.org/irefindex_resource:bio2rdf.dataset.irefindex.R3"]

def is_url(string):
    url_regex = re.compile(
        r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    )
    return re.match(url_regex, string) is not None

def execute_query_and_write_results(entity, queries, is_entity_url):
    for query in queries:
        try:
            sparql.setQuery(query)
            results = sparql.query().convert()
            types = set()
            has_results = bool(results["results"]["bindings"])  # Check if the result set is not empty

            if has_results:
                for result in results["results"]["bindings"]:
                    type_value = result.get("type", {}).get("value")
                    if type_value:
                        if type_value in ["http://www.w3.org/2002/07/owl#Class", "http://www.w3.org/2000/01/rdf-schema#Class"]:
                            types.add(entity)
                        else:
                            types.add(type_value)
                break  # Found results, no need to query other graphs

        except Exception as e:
            print(f"An error occurred while processing the entity {entity}: {e}")

    # Write results to files (outside the loop)
    if has_results and not is_entity_url:
        titles_file.write(entity + '\n')

    for type_value in types:
        types_file.write(type_value + '\n')

    if not types:
        without_type_file.write(entity + '\n')

with open('extracted_entities.csv', mode='r') as file, \
     open('10_types.txt', 'w') as types_file, \
     open('10_literals.txt', 'w') as titles_file, \
     open('10_without_type.txt', 'w') as without_type_file:

    csv_reader = csv.reader(file)
    for row in csv_reader:
        entity = row[0].strip()
        is_entity_url = is_url(entity)

        queries = []
        for graph in graphs:
            if is_entity_url:
                query = f"""
                    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
                    PREFIX dcterms: <http://purl.org/dc/terms/>

                    SELECT DISTINCT ?type  
                    FROM <{graph}>
                    WHERE {{
                        <{entity}> rdf:type ?type .
                    }}
                """
            else:
                query = f"""
                    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
                    PREFIX dcterms: <http://purl.org/dc/terms/>
                    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
                    SELECT ?type 
                    FROM <{graph}>
                    WHERE {{
                        {{
                            ?s dcterms:title "{entity}" ;
                            rdf:type ?type .
                        }}
                        UNION
                        {{
                            FILTER NOT EXISTS {{ ?s dcterms:title "{entity}" }}
                            ?s rdfs:label "{entity}" ;
                            rdf:type ?type .
                        }}
                    }}
                """
            queries.append(query)

        execute_query_and_write_results(entity, queries, is_entity_url)


# get uniques

In [41]:
# Open the types.txt file and read the types
with open('10_types.txt', 'r') as file:
    types = file.readlines()

# Use a set to get unique types
unique_types = set([type.strip() for type in types])

# Write the unique types to types.csv
with open('10_types.csv', 'w', newline='') as file:
    for type in unique_types:
        file.write(type + '\n')


30918 entities reduced to 1030 types

# get types labels:

In [16]:
from SPARQLWrapper import SPARQLWrapper, JSON
import csv

def execute_query(query):
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    titles = [result["title"]["value"] for result in results["results"]["bindings"]]
    return titles

# Set up the SPARQL endpoint connection
sparql = SPARQLWrapper("http://bio2rdf.org/sparql")  # replace with your SPARQL endpoint

# Read types from types.csv
types = []
with open('types1.csv', mode ='r') as file:
    csvReader = csv.reader(file)
    types = [row[0] for row in csvReader if row]  # assuming the type is in the first column

# Store titles or labels corresponding to types
type_titles = {}

for type_ in types:
    # Replace TYPE_URI with the actual type URI if needed
    title_query = f"""
        SELECT ?title WHERE {{
            <{type_}> <http://purl.org/dc/terms/title> ?title .
        }}
    """

    titles = execute_query(title_query)

    if not titles:
        label_query = f"""
            SELECT ?title WHERE {{
                <{type_}> <http://www.w3.org/2000/01/rdf-schema#label> ?title .
            }}
        """
        titles = execute_query(label_query)

    type_titles[type_] = titles

# Write the titles or labels to type_labels.csv
with open('types-test_labels.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Type','Title'])  # Changed this line to only include one column header

    for type_,titles in type_titles.items():
        if titles:
            for title in titles:
                writer.writerow([type_,title])  # Removed the type_ as we only want to write titles
        else:
                writer.writerow(["no"])
            
print("Titles and labels have been written to type_labels.csv")


Titles and labels have been written to type_labels.csv


In [10]:
import pandas as pd

# Load the data from the input CSV file
input_file = 'types1_labels.csv'
df = pd.read_csv(input_file)

# Get the unique rows based on all columns
unique_df = df.drop_duplicates()

# Write the unique rows to an output CSV file
output_file = 'u_types1_labels.csv'
unique_df.to_csv(output_file, index=False)

print(f"Unique values have been written to {output_file}")


Unique values have been written to u_types1_labels.csv


## properties

In [ ]:
import csv
from SPARQLWrapper import SPARQLWrapper, JSON

# List of graphs
graphs = [
"http://bio2rdf.org/sgd_resource:bio2rdf.dataset.sgd.R3",
"http://bio2rdf.org/taxonomy_resource:bio2rdf.dataset.taxonomy.R3",
"http://bio2rdf.org/homologene_resource:bio2rdf.dataset.homologene.R3",
"http://bio2rdf.org/interpro_resource:bio2rdf.dataset.interpro.R3",
"http://bio2rdf.org/bioportal_resource:bio2rdf.dataset.bioportal.R3",
"http://bio2rdf.org/clinicaltrials_resource:bio2rdf.dataset.clinicaltrials.R3",
"http://bio2rdf.org/kegg_resource:bio2rdf.dataset.kegg.R3",
"http://bio2rdf.org/pharmgkb_resource:bio2rdf.dataset.pharmgkb.R3",
"http://bio2rdf.org/hgnc_resource:bio2rdf.dataset.hgnc.R3",
"http://bio2rdf.org/mesh_resource:bio2rdf.dataset.mesh.R3",
"http://bio2rdf.org/omim_resource:bio2rdf.dataset.omim.R3",
"http://bio2rdf.org/sider_resource:bio2rdf.dataset.sider.R3",
"http://bio2rdf.org/apo_resource:bio2rdf.dataset.apo.R3",
"http://bio2rdf.org/ctd_resource:bio2rdf.dataset.ctd.R3",
"http://bio2rdf.org/go_resource:bio2rdf.dataset.go.R3",
"http://bio2rdf.org/hp_resource:bio2rdf.dataset.hp.R3",
"http://bio2rdf.org/drugbank_resource:bio2rdf.dataset.drugbank.R3",
"http://bio2rdf.org/mgi_resource:bio2rdf.dataset.mgi.R3",
"http://bio2rdf.org/goa_resource:bio2rdf.dataset.goa.R3",
"http://bio2rdf.org/ndc_resource:bio2rdf.dataset.ndc.R3",
"http://bio2rdf.org/wormbase_resource:bio2rdf.dataset.wormbase.R3",
"http://bio2rdf.org/lsr_resource:bio2rdf.dataset.lsr.R3",
"http://bio2rdf.org/affymetrix_resource:bio2rdf.dataset.affymetrix.R3",
"http://bio2rdf.org/ncbigene_resource:bio2rdf.dataset.ncbigene.R3",
"http://bio2rdf.org/eco_resource:bio2rdf.dataset.eco.R3",
"http://bio2rdf.org/irefindex_resource:bio2rdf.dataset.irefindex.R3"]

# Function to check if predicate exists in a specific graph of Bio2RDF
def check_predicate_in_graph(predicate, graph):
    sparql = SPARQLWrapper("https://bio2rdf.org/sparql")
    query = f"ASK FROM <{graph}> {{ ?s <{predicate}> ?o }}"
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    
    try:
        results = sparql.query().convert()
        return results['boolean']
    except Exception as e:
        print(f"Error querying predicate {predicate} in graph {graph}: {e}")
        return None

# Lists to hold predicates based on existence in Bio2RDF and errors
exists = []
does_not_exist = []
errors = []

# Read predicates from CSV file and check their existence in each Bio2RDF graph
with open('Logs_properties.csv', mode='r') as file:
    csvFile = csv.reader(file)
    for predicate in csvFile:
        predicate_found = False
        for graph in graphs:
            result = check_predicate_in_graph(predicate[0], graph)
            if result is True:
                exists.append(predicate[0])
                predicate_found = True
                break
            elif result is None:
                errors.append(predicate[0])
                break
        if not predicate_found and predicate[0] not in errors:
            does_not_exist.append(predicate[0])

# Write predicates that exist in Bio2RDF to a file
with open('10_Logs_properties.csv', 'w') as file:
    for predicate in exists:
        file.write(predicate + '\n')

# Write predicates that do not exist in Bio2RDF to a file
with open('10_p_not_in_bio2rdf.txt', 'w') as file:
    for predicate in does_not_exist:
        file.write(predicate + '\n')

# Print or log predicates that caused errors
print("Predicates that caused errors:")
for predicate in errors:
    print(predicate)


# Get Schema vocabularies from KG using queries

In [171]:
import requests
from SPARQLWrapper import SPARQLWrapper, JSON
import csv

# List of dataset files
dataset_files = [
"http://bio2rdf.org/sgd_resource:bio2rdf.dataset.sgd.R3",
"http://bio2rdf.org/taxonomy_resource:bio2rdf.dataset.taxonomy.R3",
"http://bio2rdf.org/homologene_resource:bio2rdf.dataset.homologene.R3",
"http://bio2rdf.org/interpro_resource:bio2rdf.dataset.interpro.R3",
"http://bio2rdf.org/bioportal_resource:bio2rdf.dataset.bioportal.R3",
"http://bio2rdf.org/clinicaltrials_resource:bio2rdf.dataset.clinicaltrials.R3",
"http://bio2rdf.org/kegg_resource:bio2rdf.dataset.kegg.R3",
"http://bio2rdf.org/pharmgkb_resource:bio2rdf.dataset.pharmgkb.R3",
"http://bio2rdf.org/hgnc_resource:bio2rdf.dataset.hgnc.R3",
"http://bio2rdf.org/mesh_resource:bio2rdf.dataset.mesh.R3",
"http://bio2rdf.org/omim_resource:bio2rdf.dataset.omim.R3",
"http://bio2rdf.org/sider_resource:bio2rdf.dataset.sider.R3",
"http://bio2rdf.org/apo_resource:bio2rdf.dataset.apo.R3",
"http://bio2rdf.org/ctd_resource:bio2rdf.dataset.ctd.R3",
"http://bio2rdf.org/go_resource:bio2rdf.dataset.go.R3",
"http://bio2rdf.org/hp_resource:bio2rdf.dataset.hp.R3",
"http://bio2rdf.org/drugbank_resource:bio2rdf.dataset.drugbank.R3",
"http://bio2rdf.org/mgi_resource:bio2rdf.dataset.mgi.R3",
"http://bio2rdf.org/goa_resource:bio2rdf.dataset.goa.R3",
"http://bio2rdf.org/ndc_resource:bio2rdf.dataset.ndc.R3",
"http://bio2rdf.org/wormbase_resource:bio2rdf.dataset.wormbase.R3",
"http://bio2rdf.org/lsr_resource:bio2rdf.dataset.lsr.R3",
"http://bio2rdf.org/affymetrix_resource:bio2rdf.dataset.affymetrix.R3",
"http://bio2rdf.org/ncbigene_resource:bio2rdf.dataset.ncbigene.R3",
"http://bio2rdf.org/eco_resource:bio2rdf.dataset.eco.R3",
"http://bio2rdf.org/irefindex_resource:bio2rdf.dataset.irefindex.R3"]

# Replace `your_sparql_endpoint` with the actual SPARQL endpoint URL
sparql_endpoint = "http://bio2rdf.org/sparql"
def execute_query(sparql_query):
    sparql = SPARQLWrapper(sparql_endpoint)
    sparql.setQuery(sparql_query)
    sparql.setReturnFormat(JSON)
    try:
        results = sparql.query().convert()
        print(f"Query returned {len(results['results']['bindings'])} results.")  # Diagnostic print
        return results
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# This will hold all distinct class types and predicates
classes_set = set()
predicates_set = set()

# Function to write results to CSV
def write_results_to_csv(file_name, results_set):
    with open(file_name, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        for item in results_set:
            writer.writerow([item])
        print(f"Wrote {len(results_set)} items to {file_name}")  # Diagnostic print

# Loop through dataset files to construct the schema
for file_url in dataset_files:
    # dataset_uri = extract_dataset_uri(file_url)
    # if dataset_uri:
        print(f"Processing {file_url}")
        # Query to find all distinct types
        query_types = f"""
        SELECT DISTINCT ?type
        FROM <{file_url}>
        WHERE {{ ?s a ?type }}
        """
        results_types = execute_query(query_types)
        for result in results_types["results"]["bindings"]:
            classes_set.add(result["type"]["value"])
        
        # Query to find all distinct predicates
        query_predicates = f"""
        SELECT DISTINCT ?p
        FROM <{file_url}>
        WHERE {{ ?s ?p ?o }}
        """
        results_predicates = execute_query(query_predicates)
        for result in results_predicates["results"]["bindings"]:
            predicates_set.add(result["p"]["value"])
        
# Write the results to CSV files
write_results_to_csv('10_Classes_schema.csv', classes_set)
write_results_to_csv('10_Predicates_schema.csv', predicates_set)

Processing http://bio2rdf.org/sgd_resource:bio2rdf.dataset.sgd.R3
Query returned 92 results.
Query returned 74 results.
Processing http://bio2rdf.org/taxonomy_resource:bio2rdf.dataset.taxonomy.R3
Query returned 11 results.
Query returned 51 results.
Processing http://bio2rdf.org/homologene_resource:bio2rdf.dataset.homologene.R3
Query returned 12 results.
Query returned 24 results.
Processing http://bio2rdf.org/interpro_resource:bio2rdf.dataset.interpro.R3
Query returned 2 results.
Query returned 13 results.
Processing http://bio2rdf.org/bioportal_resource:bio2rdf.dataset.bioportal.R3
Query returned 5 results.
Query returned 35 results.
Processing http://bio2rdf.org/clinicaltrials_resource:bio2rdf.dataset.clinicaltrials.R3
Query returned 65 results.
Query returned 168 results.
Processing http://bio2rdf.org/kegg_resource:bio2rdf.dataset.kegg.R3
Query returned 77 results.
Query returned 150 results.
Processing http://bio2rdf.org/pharmgkb_resource:bio2rdf.dataset.pharmgkb.R3
Query returned

# Get superclasses as well

In [175]:
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON

# Initialize the SPARQL wrapper with the endpoint URL
sparql = SPARQLWrapper("http://bio2rdf.org/sparql")

# Function to get the superclass of a given class
def get_superclass(class_uri):
    query = """
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT DISTINCT ?superclass
    WHERE {
      <""" + class_uri + """> rdfs:subClassOf ?superclass .
    }
    """
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    
    # Extract the superclass URIs from the query results
    superclasses = [result['superclass']['value'] for result in results["results"]["bindings"]]
    # print("s")
    # print("superclasses", superclasses)
    return superclasses

# Read the classes from the CSV file
classes_df = pd.read_csv('10_superclasses.csv')

# This assumes that the first column contains the class URIs
class_uris = classes_df.iloc[:, 0].unique()

# Get the superclasses for each class URI
superclasses_list = []
for class_uri in class_uris:
    superclasses = get_superclass(class_uri)
    superclasses_list.extend(superclasses)

# Remove duplicates and write to CSV
superclasses_df = pd.DataFrame(list(set(superclasses_list)), columns=['Superclass'])
superclasses_df.to_csv('10_sup_superclasses.csv', index=False)


# Get subclasses as well

In [ ]:
!pip install SPARQLWrapper

In [14]:
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON

# Initialize the SPARQL wrapper with the endpoint URL
sparql = SPARQLWrapper("http://bio2rdf.org/sparql")

# Function to get the superclass of a given class
def get_superclass(class_uri):
    query = """
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT DISTINCT ?subclass
    WHERE {
      ?subclass rdfs:subClassOf <""" + class_uri + """> .
    }
    """
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    
    # Extract the superclass URIs from the query results
    superclasses = [result['subclass']['value'] for result in results["results"]["bindings"]]
    # print("s")
    # print("superclasses", superclasses)
    return superclasses

# Read the classes from the CSV file
classes_df = pd.read_csv('10_subclass.csv')

# This assumes that the first column contains the class URIs
class_uris = classes_df.iloc[:, 0].unique()

# Get the superclasses for each class URI
superclasses_list = []
for class_uri in class_uris:
    superclasses = get_superclass(class_uri)
    superclasses_list.extend(superclasses)

# Remove duplicates and write to CSV
superclasses_df = pd.DataFrame(list(set(superclasses_list)), columns=['Superclass'])
superclasses_df.to_csv('10_sub_subclass.csv', index=False)


s
superclasses ['http://bio2rdf.org/wormbase_vocabulary:Change_of_expression_level-Regulatory-Interaction', 'http://bio2rdf.org/wormbase_vocabulary:Regulatory-Interaction', 'http://bio2rdf.org/wormbase_vocabulary:Change_of_localization-Regulatory-Interaction']
s
superclasses []
s
superclasses []


 ##  clean schema classes [KG]

In [172]:
import csv

# Define the input and output filenames
input_filename = '10_Classes_schema.csv'
output_filename = '10_bio2rdf_spesific_Classes_schema.csv'
count = 0
# Open the input file for reading and the output file for writing
with open(input_filename, mode='r', newline='', encoding='utf-8') as infile, \
        open(output_filename, mode='w', newline='', encoding='utf-8') as outfile:

    # Create reader and writer objects
    reader = csv.reader(infile)
    writer = csv.writer(outfile)

    # Iterate over each row in the input CSV
    for row in reader:
        # Check each cell in the row
        for value in row:
            # If the cell contains '_vocabulary' or contains 'http:' without ':', write the row to the output CSV
#             or ('http:' in value and ':' not in value.replace('http:', ''))
            if '_vocabulary' in value :
                writer.writerow(row)
                count = count + 1
                break  # We write the row once if it meets the condition and then move to the next row

print(f"Rows have been written to {output_filename} as per the conditions.")
print("count is:", count )

Rows have been written to 10_bio2rdf_spesific_Classes_schema.csv as per the conditions.
count is: 527


 ##  clean types extracted from queries

In [32]:
import csv

# Define the input and output filenames
input_filename = 'types1.csv'
output_filename = 'types_not_domain.csv'
count = 0
# Open the input file for reading and the output file for writing
with open(input_filename, mode='r', newline='', encoding='utf-8') as infile, \
        open(output_filename, mode='w', newline='', encoding='utf-8') as outfile:

    # Create reader and writer objects
    reader = csv.reader(infile)
    writer = csv.writer(outfile)

    # Iterate over each row in the input CSV
    for row in reader:
        # Check each cell in the row
        for value in row:
            # If the cell contains '_vocabulary' or contains 'http:' without ':', write the row to the output CSV
#             or ('http:' in value and ':' not in value.replace('http:', ''))
            if '_vocabulary' not in value and ('http:' in value and ':' not in value.replace('http:', '')):
                writer.writerow(row)
                count = count + 1
                break  # We write the row once if it meets the condition and then move to the next row

print(f"Rows have been written to {output_filename} as per the conditions.")
print("count is:", count )

Rows have been written to types_not_domain.csv as per the conditions.
count is: 79



## get the entities that are in kg schema but not in queries 

In [ ]:
 import csv

# Step 1: Read entities from `u_types1_labels.csv`
with open("10_bio2rdf_total_schema_elements.txt", "r") as csv_file:
    csv_reader = csv.reader(csv_file)
    entities = [row[0] for row in csv_reader]  # assuming the entity is in the first column

# Step 2: Check if they exist in `1u-labels-u-kg-types1.txt`
with open("types20.csv", "r") as txt_file:
    txt_content = txt_file.read().splitlines()

entities_in_kg = [entity for entity in entities if entity in txt_content]
entities_not_in_kg = [entity for entity in entities if entity not in txt_content]

# Step 3: Write the existing entities to `EntitiesInQueriesExist_in_KG_schema.csv`
with open("typesInQueriesExist_in_KG_schema.csv", "w", newline='') as csv_output:
    csv_writer = csv.writer(csv_output)
    for entity in entities_in_kg:
        csv_writer.writerow([entity])

# Step 4: Print the count and the entities that don't exist
print(f"Count of entities in queries that exist in KG schema: {len(entities_not_in_kg)}")
print("\nEntities that don't exist in KG schema:")
for entity in entities_not_in_kg:
    print(entity)



## get the entities that are in queries but not in kg schema

In [9]:
import csv

# Step 1: Read entities from `u_types1_labels.csv`
with open("types20.csv", "r") as csv_file:
    csv_reader = csv.reader(csv_file)
    entities = [row[0] for row in csv_reader]  # assuming the entity is in the first column

# Step 2: Check if they exist in `1u-labels-u-kg-types1.txt`
with open( "10_sup_bio2rdf_spesific_Classes_schema.csv" , "r") as txt_file:
    txt_content = txt_file.read().splitlines()

entities_in_kg = [entity for entity in entities if entity in txt_content]
entities_not_in_kg = [entity for entity in entities if entity not in txt_content]

# Step 3: Write the existing entities to `EntitiesInQueriesExist_in_KG_schema.csv`
with open("queries_types_valid_KG_schema.csv", "w", newline='') as csv_output:
    csv_writer = csv.writer(csv_output)
    for entity in entities_in_kg:
        csv_writer.writerow([entity])

# Step 4: Print the count and the entities that don't exist
print(f"Count of entities in queries that not exist in KG schema: {len(entities_not_in_kg)}")
print("\nEntities that don't exist in KG schema:")
for entity in entities_not_in_kg:
    print(entity)


Count of entities in queries that not exist in KG schema: 4

Entities that don't exist in KG schema:
http://bio2rdf.org/hgnc_vocabulary:Status
http://bio2rdf.org/drugbank_vocabulary:7cab3885cdbcb9df8c405e9c9ad10732
http://bio2rdf.org/hgnc_vocabulary:Approved
http://bio2rdf.org/atmo_vocabulary:Resource


## we delet http://bio2rdf.org/atmo_vocabulary:Resource 
because it is not in schema, it's not bio2rdf vocabulary

 ##  clean properties extracted from queries [Logs_properties.csv]

In [10]:
import csv

# Step 1: Read entities from `u_types1_labels.csv`
with open("Logs_properties.csv", "r") as csv_file:
    csv_reader = csv.reader(csv_file)
    entities = [row[0] for row in csv_reader]  # assuming the entity is in the first column

# Step 2: Check if they exist in `1u-labels-u-kg-types1.txt`
with open( "10_Predicates_schema.csv" , "r") as txt_file:
    txt_content = txt_file.read().splitlines()

entities_in_kg = [entity for entity in entities if entity in txt_content]
entities_not_in_kg = [entity for entity in entities if entity not in txt_content]

# Step 3: Write the existing entities to `EntitiesInQueriesExist_in_KG_schema.csv`
with open("queries_valid_properties_KG_schema.csv", "w", newline='') as csv_output:
    csv_writer = csv.writer(csv_output)
    for entity in entities_in_kg:
        csv_writer.writerow([entity])

# Step 4: Print the count and the entities that don't exist
print(f"Count of properties in queries that not exist in KG schema: {len(entities_not_in_kg)}")

print("\n properties that don't exist in KG schema:")
for entity in entities_not_in_kg:
    print(entity)


Count of properties in queries that not exist in KG schema: 198

 properties that don't exist in KG schema:
http://www.openlinksw.com/schemas/virtrdf#qmvftXmlIndex
http://purl.org/goodrelations/v1#includes
http://www.w3.org/ns/dcat#downloadURL
http://semanticscience.org/resource/SIO_000205
http://www.openlinksw.com/schemas/virtrdf#qmfSuperFormats
http://www.openlinksw.com/schemas/virtrdf#qmfCmpFuncName
http://www.openlinksw.com/schemas/virtrdf#qmvcAlias
http://www.openlinksw.com/schemas/virtrdf#qmfMapsOnlyNullToNull
http://www.openlinksw.com/schemas/virtrdf#qsMatchingFlags
http://www.w3.org/ns/dcat#theme
http://www.w3.org/2000/01/rdf-schema#domain
http://www.openlinksw.com/schemas/virtrdf#qmfSparqlEbvTmpl
http://www.openlinksw.com/schemas/virtrdf#qmfIidOfShortTmpl
http://purl.org/dc/terms/accrualPeriodicity
http://purl.org/dc/terms/hasPart
http://rdfs.org/ns/void#propertyPartition
http://www.openlinksw.com/schemas/virtrdf#qmfShortTmpl
http://semanticscience.org/resource/SIO_000008
http

# Get total schema element of queries

In [11]:
# Set to hold the unique elements from both files
unique_elements = set()

# Read the property_types.txt file and add its contents to the set
with open('queries_valid_properties_KG_schema.csv', 'r') as file:
    for line in file:
        unique_elements.add(line.strip())  # strip() removes any leading/trailing whitespace including newline characters

# Read the types1.csv file and add its contents to the set
with open('queries_types_valid_KG_schema.csv', 'r') as file:
    for line in file:
        unique_elements.add(line.strip())

# Write the unique elements to a new file
with open('Logs_total_schema_elements.txt', 'w') as file:
    for element in unique_elements:
        file.write(element + '\n')

print(f"{len(unique_elements)} unique elements have been written to unique_elements.txt")


1606 unique elements have been written to unique_elements.txt


# Get total schema element count of bio2rdf

In [176]:
# Set to hold the unique elements from both files
unique_elements = set()

# Read the property_types.txt file and add its contents to the set
with open('10_sup_bio2rdf_spesific_Classes_schema.csv', 'r') as file:
    for line in file:
        unique_elements.add(line.strip())  # strip() removes any leading/trailing whitespace including newline characters

# Read the types1.csv file and add its contents to the set
with open('10_Predicates_schema.csv', 'r') as file:
    for line in file:
        unique_elements.add(line.strip())

# Write the unique elements to a new file
with open('10_bio2rdf_total_schema_elements.txt', 'w') as file:
    for element in unique_elements:
        file.write(element + '\n')

print(f"{len(unique_elements)} unique elements have been written to unique_elements.txt")


1635 unique elements have been written to unique_elements.txt


In [5]:
import csv

# Step 1: Read entities from `u_types1_labels.csv`
with open("Logs_total_schema_elements.txt" , "r") as csv_file:
    csv_reader = csv.reader(csv_file)
    entities = [row[0] for row in csv_reader] # assuming the entity is in the first column

# Step 2: Check if they exist in `1u-labels-u-kg-types1.txt`
with open( "10_bio2rdf_total_schema_elements.txt"  , "r") as txt_file:
    txt_content = txt_file.read().splitlines()

entities_in_kg = [entity for entity in entities if entity in txt_content]
entities_not_in_kg = [entity for entity in entities if entity not in txt_content]


# Step 4: Print the count and the entities that don't exist
print(f"Count of entities in queries that exist in KG schema: {len(entities_not_in_kg)}")
print("\nEntities that don't exist in KG schema:")
for entity in entities_not_in_kg:
    print(entity)


Count of entities in queries that exist in KG schema: 202

Entities that don't exist in KG schema:
http://rdfs.org/ns/void#subjectsTarget
http://rdfs.org/ns/void#property
http://www.openlinksw.com/schemas/virtrdf#qmfDatatypeOfShortTmpl
http://www.openlinksw.com/schemas/virtrdf#qmvFormat
http://purl.org/dc/terms/modified
http://www.w3.org/2002/07/owl#priorVersion
http://purl.org/dc/terms/hasPart
http://bio2rdf.org/hgnc_vocabulary:Approved
http://www.bigdata.com/rdf#/features/KB/Namespace
http://www.openlinksw.com/schemas/virtrdf#qmfUriOfShortTmpl
http://www.w3.org/ns/sparql-service-description#resultFormat
http://www.openlinksw.com/schemas/virtrdf#qmfExistingShortOfUriTmpl
http://purl.org/goodrelations/v1#legalName
http://purl.org/goodrelations/v1#availableDeliveryMethods
http://www.openlinksw.com/schemas/virtrdf#qmfColumnCount
http://rdfs.org/ns/void#classes
http://www.openlinksw.com/schemas/virtrdf#qmfIsBijection
http://www.w3.org/1999/02/22-rdf-syntax-ns#_4
http://purl.org/ontology/w

In [26]:
import csv

# Read entities from `bio2rdf_total_schema_elements.txt`
with open("bio2rdf_total_schema_elements.txt", "r") as csv_file:
    csv_reader = csv.reader(csv_file)
    entities = [row[0] for row in csv_reader]  # assuming the entity is in the first column

# Check if they exist in `Logs_total_schema_elements.txt`
with open("Logs_total_schema_elements.txt", "r") as txt_file:
    txt_content = txt_file.read().splitlines()

# Determine which entities are not in the knowledge graph
entities_not_in_kg = [entity for entity in entities if entity not in txt_content]

# Step 4: Print the count and the entities that don't exist
# We will sort the entities based on whether they contain '_vocabulary:Resource' or '_vocabulary'
resource_entities = [entity for entity in entities_not_in_kg if '_vocabulary:Resource' in entity]
vocabulary_entities = [entity for entity in entities_not_in_kg if '_vocabulary' in entity and entity not in resource_entities]
other_entities = [entity for entity in entities_not_in_kg if entity not in resource_entities and entity not in vocabulary_entities]

# Now we combine the lists, keeping the desired order
sorted_entities_not_in_kg = vocabulary_entities + resource_entities + other_entities

print(f"Count of entities not in KG schema: {len(sorted_entities_not_in_kg)}")
print("\nEntities that don't exist in KG schema:")
for entity in sorted_entities_not_in_kg:
    print(entity)
# 10_Subclass_all.csv

Count of entities not in KG schema: 54

Entities that don't exist in KG schema:
http://bio2rdf.org/clinicaltrials_vocabulary:limitations-and-caveats
http://bio2rdf.org/clinicaltrials_vocabulary:ci-upper-limit
http://bio2rdf.org/clinicaltrials_vocabulary:lower-limit
http://bio2rdf.org/obo_vocabulary:Entity
http://bio2rdf.org/kegg_vocabulary:Reversible-Reaction
http://bio2rdf.org/clinicaltrials_vocabulary:ci-upper-limit-na-comment
http://bio2rdf.org/clinicaltrials_vocabulary:upper-limit
http://bio2rdf.org/clinicaltrials_vocabulary:ci-lower-limit
http://bio2rdf.org/kegg_vocabulary:Irreversible-Reaction
http://bio2rdf.org/broad-lincrna_vocabulary:Resource
http://bio2rdf.org/ed_vocabulary:Resource
http://bio2rdf.org/ligandbox_vocabulary:Resource
http://bio2rdf.org/ncbi-proteinid_vocabulary:Resource
http://bio2rdf.org/solgenomics_vocabulary:Resource
http://bio2rdf.org/luo_lincrna_vocabulary:Resource
http://bio2rdf.org/signalp_vocabulary:Resource
http://bio2rdf.org/utoronto_vocabulary:Resourc

In [18]:
import csv

# Read entities from `10_Subclass_all.csv`
entities = []
with open("10_Subclass_all.csv", "r") as csv_file:
    csv_reader = csv.reader(csv_file)
    for row in csv_reader:
        if row:  # Check if row is not empty
            entities.append(row[0])  # Assuming the entity is in the first column

# Check if they exist in `10_bio2rdf_total_schema_elements.txt`
with open("10_bio2rdf_total_schema_elements.txt", "r") as txt_file:
    txt_content = txt_file.read().splitlines()

# Determine which entities are not in the knowledge graph
entities_not_in_kg = [entity for entity in entities if entity not in txt_content]

print(f"Count of entities not in KG schema: {len(entities_not_in_kg)}")
print("\nEntities that don't exist in KG schema:")
for entity in entities_not_in_kg:
    print(entity)


Count of entities not in KG schema: 0

Entities that don't exist in KG schema:
